# Libero

In [39]:
!rm -r /kaggle/working/LIBERO/libero/datasets/libero_spatial

In [ ]:
import sys
import os
from unittest.mock import MagicMock

# 1. Fix PyTorch Bugs & Crashers
os.environ["TORCH_FSDP_NO_WARN_CIRCULAR_DEPENDENCY"] = "1"
os.environ["TORCH_ALLOW_TF32"] = "1"

# 2. Mock Matplotlib (Critical to prevent the 0x10 crash later)
mock_mpl = MagicMock()
for module in ["matplotlib", "matplotlib.pyplot", "matplotlib.cm", "matplotlib.colors", "matplotlib.transforms", "matplotlib.ticker", "matplotlib.path", "matplotlib._path"]:
    sys.modules[module] = mock_mpl

# 3. Clone LIBERO
if not os.path.exists("LIBERO"):
    print("⬇️ Cloning LIBERO...")
    !git clone https://github.com/Lifelong-Robot-Learning/LIBERO.git
else:
    print("✓ LIBERO already cloned")

# 4. Smart Install (Skips heavy/conflicting libs)
print("\n📦 Installing LIBERO requirements (Lightweight Mode)...")
# We filter out numpy, torch, etc. to use Kaggle's pre-installed fast versions
!grep -v -E "numpy|matplotlib|tokenizers|wandb|transformers|opencv-python|torch" /kaggle/working/LIBERO/requirements.txt > /kaggle/working/LIBERO/filtered_requirements.txt
!pip install -r /kaggle/working/LIBERO/filtered_requirements.txt --no-deps

# 5. Download Dataset (Visible Progress!)
# Removed "> /dev/null" so you can see it working.
print("\n🌍 Downloading LIBERO-Spatial (This make take 1-2 mins)...")
!echo n | python /kaggle/working/LIBERO/benchmark_scripts/download_libero_datasets.py --datasets libero_spatial --use-huggingface

print("\n✅ BASE SETUP COMPLETE.")

In [7]:
print("📦 Installing extra project tools...")

# 1. Install missing tools
!pip install einops robomimic

# 2. Fix Tokenizers (Specific version required by some models)
!pip install tokenizers==0.15.2

# 3. MuJoCo (Only if needed, usually safe)
!pip install mujoco==2.3.7

# NOTE: We SKIP 'wandb', 'transformers', and 'numba' downgrade because 
# Kaggle already has better versions installed.

print("✅ EXTRA TOOLS INSTALLED.")

📦 Installing extra project tools...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.5/217.5 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 33.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/

# Libraries

In [8]:
print("Hello from trm-on-robotic.ipynb")

import torch
import os
import torch.nn as nn
import numpy as np
import h5py
import einops
from torchvision import models
from torchvision.models import resnet18, ResNet18_Weights
from torchvision import transforms

# Verifica disponibilità GPU
print(f"PyTorch Version: {torch.__version__}")
if torch.cuda.is_available():
    print(f"✅ GPU Disponibile: {torch.cuda.get_device_name(0)}")
    device = torch.device("cuda")
else:
    print("⚠️ ATTENZIONE: GPU non rilevata. Vai su 'Settings' > 'Accelerator' e seleziona GPU P100 o T4.")
    device = torch.device("cpu")

# Verifica Einops (lo useremo molto nel modello ricorsivo)
print(f"Einops installato correttamente.")

# Configurazione base per riproducibilità (importante per la tesi)
def set_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)
print("✅ Ambiente pronto per il modello TinyRecursive.")

Hello from trm-on-robotic.ipynb
PyTorch Version: 2.6.0+cu124
✅ GPU Disponibile: Tesla P100-PCIE-16GB
Einops installato correttamente.
✅ Ambiente pronto per il modello TinyRecursive.


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import h5py
# import matplotlib.pyplot as plt
from pathlib import Path
import cv2
from typing import Dict, List, Tuple, Optional, Any
from dataclasses import dataclass, asdict, replace
import json
import wandb
import optuna
from datetime import datetime

# Setup device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Seed per riproducibilità
def set_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# =========================
# Configuration utilities
# =========================

@dataclass
class TrainingConfig:
    """Container strutturato per gli iperparametri di training/model."""

    lr: float = 3e-4
    hidden_dim: int = 256
    num_recursions: int = 8
    epochs: int = 20
    batch_size: int = 64
    weight_decay: float = 1e-4
    grad_clip: Optional[float] = 1.0
    sched_T0: Optional[int] = None
    sched_T_mult: int = 1
    lr_min: float = 1e-6
    warmup_epochs: int = 3
    early_stop_patience: Optional[int] = None
    save_path: str = 'best_model.pt'
    use_pretrained_encoder: bool = True
    freeze_backbone: bool = True
    augmentation: bool = False
    dropout: float = 0.1
    encoder_dropout: float = 0.1

    def to_dict(self) -> Dict[str, Any]:
        return asdict(self)

    def label(self) -> str:
        return f"lr{self.lr}_h{self.hidden_dim}_rec{self.num_recursions}_bs{self.batch_size}"


@dataclass
class HyperparameterSearchSpace:
    lr: List[float]
    hidden_dim: List[int]
    num_recursions: List[int]
    batch_size: List[int]
    weight_decay: List[float]
    pretrained_encoder: List[bool]
    freeze_backbone: List[bool]
    augmentation: List[bool]
    dropout: List[float]

    def as_optuna_space(self) -> Dict[str, List[Any]]:
        return {
            'lr': self.lr,
            'hidden_dim': self.hidden_dim,
            'num_recursions': self.num_recursions,
            'batch_size': self.batch_size,
            'weight_decay': self.weight_decay,
            'pretrained_encoder': self.pretrained_encoder,
            'freeze_backbone': self.freeze_backbone,
            'augmentation': self.augmentation,
            'dropout': self.dropout,
        }

# Numero 1
def default_search_space() -> HyperparameterSearchSpace:
    """Restituisce lo spazio di ricerca richiesto dall'utente."""

    return HyperparameterSearchSpace(
        lr=[1e-5, 1e-6, 1e-4, 1e-3],
        hidden_dim=[128, 256, 512],
        num_recursions=[8, 12, 16],
        batch_size=[32, 64, 128, 256, 512],
        weight_decay=[0.1, 0.5, 1.0],
        pretrained_encoder=[False],
        freeze_backbone=[False],
        augmentation=[True, False],
        dropout=[0.1, 0.3, 0.5, 0.7],
    )

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Using device: cuda


In [ ]:
import numpy as np
import h5py
import sys
from pathlib import Path
from PIL import Image
from IPython.display import display

# --- HELPER FUNCTIONS (Kept largely the same) ---

def load_images_robust(dataset):
    """
    Carica immagini da dataset HDF5 usando metodo robusto.
    """
    shape = dataset.shape
    
    # METODO 1: Lettura diretta uint8
    try:
        buffer = np.empty(shape, dtype=np.uint8)
        dataset.read_direct(buffer)
        return buffer
    except Exception:
        pass
    
    # METODO 2: Float32 -> Uint8
    try:
        buffer = np.empty(shape, dtype=np.float32)
        dataset.read_direct(buffer)
        if buffer.max() <= 1.0:
            buffer = (buffer * 255).astype(np.uint8)
        else:
            buffer = np.clip(buffer, 0, 255).astype(np.uint8)
        return buffer
    except Exception:
        pass
    
    # METODO 3: Float64 -> Uint8
    try:
        buffer = np.empty(shape, dtype=np.float64)
        dataset.read_direct(buffer)
        if buffer.max() <= 1.0:
            buffer = (buffer * 255).astype(np.uint8)
        else:
            buffer = np.clip(buffer, 0, 255).astype(np.uint8)
        return buffer
    except Exception:
        pass

    # METODO 4: Fallback bytes
    try:
        buffer = np.empty(shape, dtype=np.uint8)
        dataset.id.read(h5py.h5s.ALL, h5py.h5s.ALL, buffer)
        return buffer
    except Exception as e:
        raise RuntimeError(f"Impossibile leggere il dataset: {e}")

def load_actions_robust(dataset):
    """
    Carica azioni da dataset HDF5.
    """
    shape = dataset.shape
    try:
        buffer = np.empty(shape, dtype=np.float32)
        dataset.read_direct(buffer)
        return buffer
    except Exception:
        pass
    
    try:
        buffer = np.empty(shape, dtype=np.float64)
        dataset.read_direct(buffer)
        return buffer.astype(np.float32)
    except Exception as e:
        raise RuntimeError(f"Impossibile leggere le azioni: {e}")

# --- MODIFIED EXPLORATION FUNCTION (No Matplotlib) ---

def explore_libero_dataset(data_path: Path):
    
    # Trova file
    hdf5_files = list(data_path.glob('**/*.hdf5'))
    
    if not hdf5_files:
        print(f"⚠️ Nessun file HDF5 trovato in {data_path}")
        return []
    
    print(f"✅ Trovati {len(hdf5_files)} file HDF5")
    
    # Analizza il primo file
    demo_file = hdf5_files[0]
    print(f"\n📄 Analizzando: {demo_file.name}")
    
    try:
        with h5py.File(demo_file, 'r') as f:
            if 'data' not in f:
                print("⚠️ Chiave 'data' non trovata")
                return hdf5_files
            
            data_group = f['data']
            demo_keys = list(data_group.keys())
            first_demo_key = demo_keys[0]
            demo_0 = data_group[first_demo_key]
            
            imgs = None
            
            # 1. Caricamento Immagini
            if 'obs' in demo_0:
                obs_group = demo_0['obs']
                
                # Strategia di ricerca chiave immagine
                image_keys = ['agentview_rgb', 'agentview_image', 'rgb', 'image', 'robot0_eye_in_hand_image']
                img_key = next((k for k in image_keys if k in obs_group), None)
                
                # Fallback ricerca generica
                if img_key is None:
                    img_key = next((k for k in obs_group.keys() if 'rgb' in k.lower() or 'image' in k.lower()), None)
                
                if img_key:
                    print(f"\n🖼️ Usando chiave immagini: '{img_key}'")
                    try:
                        imgs = load_images_robust(obs_group[img_key])
                        print(f"  ✅ Immagini caricate: {imgs.shape}")
                    except Exception as e:
                        print(f"  ❌ Errore immagini: {e}")
            
            # 2. Caricamento Azioni
            if 'actions' in demo_0:
                try:
                    actions = load_actions_robust(demo_0['actions'])
                    print(f"\n🎮 Azioni caricate: {actions.shape}")
                    print(f"  Range: [{actions.min():.3f}, {actions.max():.3f}]")
                except Exception as e:
                    print(f"  ❌ Errore azioni: {e}")

            # 3. VISUALIZZAZIONE (Senza Matplotlib)
            if imgs is not None and len(imgs) > 0:
                print("\n🎬 Visualizzazione frame esempio (PIL/IPython):")
                
                num_frames = min(4, len(imgs))
                indices = np.linspace(0, len(imgs) - 1, num_frames, dtype=int)
                
                for idx in indices:
                    img_array = imgs[idx]
                    
                    # Se l'immagine è float [0,1], converti a uint8
                    if img_array.dtype != np.uint8:
                         img_array = (np.clip(img_array, 0, 1) * 255).astype(np.uint8)
                    
                    # Crea immagine PIL
                    pil_img = Image.fromarray(img_array)
                    
                    # (Opzionale) Resize per non occupare troppo spazio
                    # pil_img = pil_img.resize((128, 128))
                    
                    print(f"--- Frame {idx} ---")
                    display(pil_img)
            else:
                print("\n⚠️ Nessuna immagine valida da visualizzare")

    except Exception as e:
        print(f"Errore critico durante l'apertura del file: {e}")
    
    return hdf5_files

# Esegui
hdf5_files = explore_libero_dataset(Path('/kaggle/working/LIBERO/libero/datasets/libero_spatial'))

In [ ]:
class LIBERODataset(Dataset):
    """
    Dataset per dimostrazioni LIBERO
    
    Carica osservazioni visive e azioni da file HDF5.
    Supporta data augmentation e normalizzazione.
    Gestisce automaticamente problemi di dtype non standard nei file HDF5.
    
    Supporta demo-level split: invece di dividere i file, divide le demo
    all'interno di ciascun file (es. 80% train, 20% val per ogni file).
    """
    
    def __init__(
        self,
        hdf5_files: List[Path],
        sequence_length: int = 1,
        image_size: Tuple[int, int] = (128, 128),
        normalize_actions: bool = True,
        augmentation: bool = False,
        max_demos_per_task: Optional[int] = None,
        demo_split_ratio: float = 0.8,
        is_train: bool = True,
        action_stats: Optional[Dict] = None
    ):
        """
        Args:
            hdf5_files: lista di path ai file HDF5
            sequence_length: lunghezza delle sequenze (1 = single-step prediction)
            image_size: dimensioni delle immagini
            normalize_actions: se True, normalizza le azioni con z-score
            augmentation: se True, applica data augmentation
            max_demos_per_task: limite massimo di demo per task (per debugging)
            demo_split_ratio: percentuale di demo per training (default 0.8 = 80%)
            is_train: se True, usa le prime demo_split_ratio% demo; se False, usa il resto
            action_stats: statistiche azioni pre-calcolate (per validation set)
        """
        self.hdf5_files = hdf5_files
        self.sequence_length = sequence_length
        self.image_size = (int(image_size[0]), int(image_size[1]))
        self.augmentation = augmentation and is_train
        self.normalize_actions = normalize_actions
        self.demo_split_ratio = demo_split_ratio
        self.is_train = is_train
        
        # Carica tutti i dati in memoria (assumendo dataset gestibile)
        self.data = []
        self.action_stats = action_stats if action_stats is not None else {'mean': None, 'std': None}
        self.samples: List[Tuple[int, int]] = []  # (demo_idx, start_idx)
        
        split_name = "TRAIN" if is_train else "VAL"
        print(f"Loading {len(hdf5_files)} HDF5 files for {split_name} (demo split: {demo_split_ratio:.0%})...")
        all_actions = []
        
        for hdf5_file in hdf5_files:
            try:
                with h5py.File(hdf5_file, 'r') as f:
                    if 'data' not in f:
                        print(f"⚠️ 'data' key not found in {hdf5_file.name}, skipping...")
                        continue
                    
                    demo_keys = list(f['data'].keys())
                    
                    # Limita numero di demo se richiesto
                    if max_demos_per_task is not None:
                        demo_keys = demo_keys[:max_demos_per_task]
                    
                    # Demo-level split: seleziona subset di demo in base a is_train
                    n_demos = len(demo_keys)
                    n_train_demos = int(n_demos * demo_split_ratio)
                    
                    if is_train:
                        # Training: prime n_train_demos demo
                        selected_demo_keys = demo_keys[:n_train_demos]
                    else:
                        # Validation: demo rimanenti
                        selected_demo_keys = demo_keys[n_train_demos:]
                    
                    if len(selected_demo_keys) == 0:
                        print(f"⚠️ No demos selected from {hdf5_file.name} for {split_name}, skipping...")
                        continue
                    
                    for demo_key in selected_demo_keys:
                        try:
                            demo = f[f'data/{demo_key}']
                            
                            # Trova la chiave delle immagini
                            obs_group = demo['obs']
                            img_key = self._find_image_key(obs_group)
                            
                            if img_key is None:
                                print(f"⚠️ No image key found in {hdf5_file.name}/{demo_key}, skipping...")
                                continue
                            
                            # Carica osservazioni con metodo robusto
                            obs = self._load_images_robust(obs_group[img_key])
                            
                            # Carica azioni con metodo robusto
                            actions = self._load_actions_robust(demo['actions'])
                            
                            # Verifica che obs e actions abbiano lunghezze compatibili
                            min_len = min(len(obs), len(actions))
                            if min_len < self.sequence_length:
                                print(f"⚠️ Demo too short ({min_len} < {self.sequence_length}), skipping...")
                                continue
                            
                            obs = obs[:min_len]
                            actions = actions[:min_len]
                            
                            # Aggiungi alla lista
                            self.data.append({
                                'observations': obs,
                                'actions': actions
                            })
                            
                            all_actions.append(actions)
                            
                        except Exception as e:
                            print(f"⚠️ Error loading demo {demo_key} from {hdf5_file.name}: {e}")
                            continue
                            
            except Exception as e:
                print(f"❌ Error opening file {hdf5_file}: {e}")
                continue
        
        print(f"✅ Loaded {len(self.data)} demonstrations for {split_name}")
        
        if len(self.data) == 0:
            raise ValueError(f"No valid demonstrations loaded for {split_name}! Check your data files.")
        
        # Calcola statistiche azioni per normalizzazione (solo per training set o se non fornite)
        if self.normalize_actions and len(all_actions) > 0 and action_stats is None:
            all_actions_concat = np.concatenate(all_actions, axis=0)
        
            mean = all_actions_concat.mean(axis=0).astype(np.float32)
            std  = all_actions_concat.std(axis=0).astype(np.float32)
        
            # ⚠️ Floor di sicurezza: evita std troppo piccole che esplodono la normalizzazione
            std_clipped = np.clip(std, 0.1, None)
        
            # Log dettagliato
            print(f"📊 Action statistics computed from {split_name} set:")
            print(f"   Mean:        {np.round(mean, 3)}")
            print(f"   Std (raw):   {np.round(std, 3)}")
            print(f"   Std (clipped to >=0.1): {np.round(std_clipped, 3)}")
        
            self.action_stats['mean'] = mean
            self.action_stats['std']  = std_clipped
        
        elif action_stats is not None:
            print(f"📊 Using provided action statistics")
            self.action_stats = {
                'mean': action_stats['mean'].astype(np.float32),
                'std':  np.clip(action_stats['std'], 0.1, None).astype(np.float32)
            }

        # Costruisci indice delle transizioni per accesso O(1)
        self.samples = self._build_sample_index()
        print(f"📦 Generated {len(self.samples)} transitions for {split_name}")

    def _find_image_key(self, obs_group) -> Optional[str]:
        """Trova la chiave corretta per le immagini nel gruppo obs"""
        # Lista di possibili chiavi per le immagini (in ordine di priorità)
        possible_keys = [
            'agentview_rgb',
            'agentview_image', 
            'rgb',
            'image',
            'robot0_eye_in_hand_image',
            'frontview_image',
            'sideview_image'
        ]
        
        obs_keys = list(obs_group.keys())
        
        # Prima cerca chiavi note
        for key in possible_keys:
            if key in obs_keys:
                return key
        
        # Poi cerca qualsiasi chiave che contiene 'rgb' o 'image'
        for key in obs_keys:
            if 'rgb' in key.lower() or 'image' in key.lower():
                return key
        
        return None
    
    def _load_images_robust(self, dataset) -> np.ndarray:
        """
        Carica immagini da dataset HDF5 usando metodo robusto che bypassa problemi dtype.
        """
        shape = dataset.shape
        
        # METODO 1: Prova lettura diretta come uint8
        try:
            buffer = np.empty(shape, dtype=np.uint8)
            dataset.read_direct(buffer)
            return buffer
        except Exception:
            pass
        
        # METODO 2: Prova come float32 e converti
        try:
            buffer = np.empty(shape, dtype=np.float32)
            dataset.read_direct(buffer)
            if buffer.max() <= 1.0:
                buffer = (buffer * 255).astype(np.uint8)
            else:
                buffer = np.clip(buffer, 0, 255).astype(np.uint8)
            return buffer
        except Exception:
            pass
        
        # METODO 3: Prova come float64
        try:
            buffer = np.empty(shape, dtype=np.float64)
            dataset.read_direct(buffer)
            if buffer.max() <= 1.0:
                buffer = (buffer * 255).astype(np.uint8)
            else:
                buffer = np.clip(buffer, 0, 255).astype(np.uint8)
            return buffer
        except Exception:
            pass
        
        # METODO 4: Lettura raw
        try:
            buffer = np.empty(shape, dtype=np.uint8)
            dataset.id.read(h5py.h5s.ALL, h5py.h5s.ALL, buffer)
            return buffer
        except Exception as e:
            raise RuntimeError(f"Cannot read image dataset: {e}")
    
    def _load_actions_robust(self, dataset) -> np.ndarray:
        """
        Carica azioni da dataset HDF5 usando metodo robusto.
        """
        shape = dataset.shape
        
        # METODO 1: Prova lettura diretta come float32
        try:
            buffer = np.empty(shape, dtype=np.float32)
            dataset.read_direct(buffer)
            return buffer
        except Exception:
            pass
        
        # METODO 2: Prova come float64 e converti
        try:
            buffer = np.empty(shape, dtype=np.float64)
            dataset.read_direct(buffer)
            return buffer.astype(np.float32)
        except Exception as e:
            raise RuntimeError(f"Cannot read actions dataset: {e}")
    
    def _build_sample_index(self) -> List[Tuple[int, int]]:
        """Pre-calcola gli indici (demo_idx, start_idx) per ogni transizione"""
        indices: List[Tuple[int, int]] = []
        for demo_idx, demo in enumerate(self.data):
            demo_transitions = len(demo['observations']) - self.sequence_length + 1
            if demo_transitions <= 0:
                continue
            indices.extend((demo_idx, start) for start in range(demo_transitions))
        if not indices:
            raise ValueError("Dataset index is empty after preprocessing")
        return indices

    def __len__(self) -> int:
        """Numero totale di transizioni disponibili"""
        return len(self.samples)
    
    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        """
        Restituisce una transizione (osservazione, azione).
        
        Args:
            idx: indice della transizione
            
        Returns:
            Dict con 'observations' e 'actions' come tensori
        """
        demo_idx, start_idx = self.samples[idx]
        demo = self.data[demo_idx]
        end_idx = start_idx + self.sequence_length

        obs = demo['observations'][start_idx:end_idx].copy()
        actions = demo['actions'][start_idx:end_idx].copy()

        # Preprocessing
        obs = self._preprocess_obs(obs)
        actions = self._preprocess_actions(actions)

        # Per single-step prediction, restituisci solo primo elemento
        if self.sequence_length == 1:
            obs = obs[0]
            actions = actions[0]

        # Converti HWC -> CHW per PyTorch
        if obs.ndim == 3:  # Single image: (H, W, C) -> (C, H, W)
            obs = np.transpose(obs, (2, 0, 1))
        elif obs.ndim == 4:  # Sequence: (T, H, W, C) -> (T, C, H, W)
            obs = np.transpose(obs, (0, 3, 1, 2))

        return {
            'observations': torch.from_numpy(obs).float(),
            'actions': torch.from_numpy(actions).float()
        }
    
    def _preprocess_obs(self, obs: np.ndarray) -> np.ndarray:
        """Preprocessing delle osservazioni"""
        processed = []
        target_h, target_w = self.image_size
        for img in obs:
            if img.shape[0] != target_h or img.shape[1] != target_w:
                img = cv2.resize(img, (target_w, target_h), interpolation=cv2.INTER_AREA)
            processed.append(img)
        obs = np.stack(processed, axis=0)

        # Normalizza [0, 255] -> [0, 1]
        obs = obs.astype(np.float32) / 255.0

        # Data augmentation (se abilitato)
        if self.augmentation:
            obs = self._augment_obs(obs)
        
        return obs
    def _augment_obs(self, obs: np.ndarray) -> np.ndarray:
        """Data augmentation per osservazioni"""
        # Color jitter (brightness)
        if np.random.rand() < 0.5:
            brightness = np.random.uniform(0.8, 1.2)
            obs = np.clip(obs * brightness, 0, 1)
        
        # Contrast adjustment
        if np.random.rand() < 0.3:
            contrast = np.random.uniform(0.8, 1.2)
            mean = obs.mean(axis=(1, 2), keepdims=True)
            obs = np.clip((obs - mean) * contrast + mean, 0, 1)
        
        # Random crop (con padding)
        if np.random.rand() < 0.3:
            crop_ratio = np.random.uniform(0.85, 0.95)
            crop_size_h = int(self.image_size[0] * crop_ratio)
            crop_size_w = int(self.image_size[1] * crop_ratio)
            
            start_y = np.random.randint(0, self.image_size[0] - crop_size_h + 1)
            start_x = np.random.randint(0, self.image_size[1] - crop_size_w + 1)
            
            cropped = []
            for img in obs:
                img_crop = img[start_y:start_y+crop_size_h, start_x:start_x+crop_size_w]
                img_resized = cv2.resize(img_crop, (self.image_size[1], self.image_size[0]))
                cropped.append(img_resized)
            obs = np.stack(cropped)
        
        return obs
    
    def _preprocess_actions(self, actions: np.ndarray) -> np.ndarray:
        """Preprocessing delle azioni con normalizzazione z-score"""
        actions = actions.astype(np.float32)
        if self.action_stats['mean'] is not None:
            actions = (actions - self.action_stats['mean']) / self.action_stats['std']
        
        return actions
    
    def get_action_stats(self) -> Dict[str, np.ndarray]:
        """Restituisce le statistiche delle azioni per denormalizzazione"""
        return self.action_stats.copy()
    
    def denormalize_actions(self, actions: np.ndarray) -> np.ndarray:
        """Denormalizza le azioni per l'esecuzione nel simulatore"""
        if self.action_stats['mean'] is not None:
            return actions * self.action_stats['std'] + self.action_stats['mean']
        return actions

In [13]:
class PretrainedVisualEncoder(nn.Module):
    """Visual encoder basato su ResNet18 con testa adattiva."""

    def __init__(self, hidden_dim: int = 256, freeze_backbone: bool = True, dropout: float = 0.1):
        super().__init__()

        self.hidden_dim = hidden_dim

        resnet = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

        if freeze_backbone:
            for param in resnet.parameters():
                param.requires_grad = False

        self.backbone = nn.Sequential(*list(resnet.children())[:-1])

        self.adapter = nn.Sequential(
            nn.Linear(512, 512),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(512, hidden_dim)
        )
        self.ln = nn.LayerNorm(hidden_dim)

        self.register_buffer('mean', torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1))
        self.register_buffer('std', torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1))

        self._init_weights()

    def _init_weights(self):
        for m in self.adapter.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.float()

        if x.shape[-1] != 224 or x.shape[-2] != 224:
            x = nn.functional.interpolate(x, size=(224, 224), mode='bilinear', align_corners=False)

        x = (x - self.mean) / self.std
        features = self.backbone(x).flatten(start_dim=1)
        output = self.adapter(features)
        return self.ln(output)


class VisualEncoder(nn.Module):
    def __init__(self, obs_shape: Tuple[int, int, int] = (3, 128, 128), hidden_dim: int = 256, dropout: float = 0.1):
        super().__init__()

        c, _, _ = obs_shape
        self.conv = nn.Sequential(
            nn.Conv2d(c, 32, 3, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, 3, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, 3, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1))
        )

        self.head = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout)
        )

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv(x)
        return self.head(x)


class RecursiveBlock(nn.Module):
    """
    Blocco ricorsivo del TRM.
    Implementa il ragionamento iterativo con self-attention e MLP.
    """
    
    def __init__(self, hidden_dim=256, num_heads=4, dropout=0.1):
        super().__init__()
        
        # Self-attention per ragionamento
        self.attention = nn.MultiheadAttention(
            hidden_dim,
            num_heads,
            dropout=dropout,
            batch_first=True
        )
        self.norm1 = nn.LayerNorm(hidden_dim)
        self.dropout1 = nn.Dropout(dropout)
        
        # MLP per trasformazione
        self.mlp = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim * 4),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim * 4, hidden_dim),
            nn.Dropout(dropout)
        )
        self.norm2 = nn.LayerNorm(hidden_dim)
    
    def forward(self, h, x_cond):
        """
        Args:
            h: (B, D) hidden state corrente
            x_cond: (B, D) conditioning dall'input
        Returns:
            (B, D) nuovo hidden state
        """
        # Combina hidden state e conditioning
        combined = h + x_cond
        combined = combined.unsqueeze(1)  # (B, 1, D) per attention
        
        # Self-attention con residual
        attn_out, _ = self.attention(combined, combined, combined)
        combined = self.norm1(combined + self.dropout1(attn_out))
        
        # MLP con residual
        mlp_out = self.mlp(combined)
        h_new = self.norm2(combined + mlp_out)
        
        return h_new.squeeze(1)  # (B, D)


class TRMPolicy(nn.Module):
    """Policy con heads separati e component attention"""
    def __init__(self, obs_shape=(3, 128, 128), action_dim=7, hidden_dim=256, num_heads=4, num_recursions=8, dropout=0.1, use_pretrained_encoder=True, freeze_backbone=True, encoder_dropout=0.1):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_recursions = num_recursions
        self.obs_shape = obs_shape
        self.use_pretrained_encoder = use_pretrained_encoder
        # Encoder
        if use_pretrained_encoder:
            self.encoder = PretrainedVisualEncoder(hidden_dim=hidden_dim, freeze_backbone=freeze_backbone, dropout=encoder_dropout)
        else:
            self.encoder = VisualEncoder(obs_shape=obs_shape, hidden_dim=hidden_dim, dropout=encoder_dropout)
        # Recursive block
        self.recursive_block = RecursiveBlock(hidden_dim, num_heads, dropout)
        # Component attention
        self.component_attention = nn.MultiheadAttention(hidden_dim, num_heads, batch_first=True)
        # Heads separati
        self.head_pos = nn.Sequential(
            nn.LayerNorm(hidden_dim),
            nn.Linear(hidden_dim, 32),
            nn.GELU(),
            nn.Linear(32, 3)
        )
        self.head_rot = nn.Sequential(
            nn.LayerNorm(hidden_dim),
            nn.Linear(hidden_dim, 32),
            nn.GELU(),
            nn.Linear(32, 3)
        )
        self.head_grip = nn.Sequential(
            nn.LayerNorm(hidden_dim),
            nn.Linear(hidden_dim, 16),
            nn.GELU(),
            nn.Linear(16, 1)
        )
        # Xavier uniform init
        for m in [self.head_pos, self.head_rot, self.head_grip]:
            for layer in m:
                if isinstance(layer, nn.Linear):
                    nn.init.xavier_uniform_(layer.weight)
                    nn.init.zeros_(layer.bias)
    def forward(self, obs):
        x_cond = self.encoder(obs)
        h = x_cond.clone()
        for _ in range(self.num_recursions):
            h = self.recursive_block(h, x_cond)
        # Component attention: stack and attend
        h_stack = torch.stack([h, h, h], dim=1)  # (B, 3, D)
        h_attn, _ = self.component_attention(h_stack, h_stack, h_stack)
        # Heads
        pos = self.head_pos(h_attn[:,0])
        rot = self.head_rot(h_attn[:,1])
        grip = self.head_grip(h_attn[:,2])
        return torch.cat([pos, rot, grip], dim=-1)

In [30]:
def build_policy_from_config(config: TrainingConfig, obs_shape: Tuple[int, int, int] = (3, 128, 128)) -> TRMPolicy:
    """Costruisce una TRMPolicy coerente con il TrainingConfig."""

    return TRMPolicy(
        obs_shape=obs_shape,
        action_dim=7,
        hidden_dim=config.hidden_dim,
        num_recursions=config.num_recursions,
        dropout=config.dropout,
        use_pretrained_encoder=config.use_pretrained_encoder,
        freeze_backbone=config.freeze_backbone,
        encoder_dropout=config.encoder_dropout
    )

class BehaviorCloningTrainer:
    """Trainer per Behavior Cloning con configurazione strutturata."""

    def __init__(
        self,
        model: nn.Module,
        train_loader: DataLoader,
        val_loader: DataLoader,
        config: TrainingConfig,
        device: torch.device,
        use_wandb: bool = False
    ):
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.config = config
        self.device = device
        self.use_wandb = use_wandb
        self.steps_per_epoch = max(len(train_loader), 1)
        self.optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=config.lr,
            weight_decay=config.weight_decay,
            amsgrad=True
        )
        self.scheduler = torch.optim.lr_scheduler.OneCycleLR(
            self.optimizer,
            max_lr=config.lr,
            steps_per_epoch=self.steps_per_epoch,
            epochs=config.epochs
        )
        self.use_amp = (self.device.type == 'cuda')
        self.scaler = torch.cuda.amp.GradScaler(enabled=self.use_amp)
        self.grad_clip = config.grad_clip
        self.early_stop_patience = config.early_stop_patience
        self._epochs_no_improve = 0
        self.best_val_loss = float('inf')
        self.best_model_path = config.save_path
    def train(self):
        """Training loop completo"""
        
        for epoch in range(self.config.epochs):
            # Training
            train_metrics = self._train_epoch(epoch)
            
            # Validation
            val_metrics = self._validate_epoch(epoch)
            
            # Logging exaustive
            self._log_metrics(epoch, train_metrics, val_metrics)

            # print(f"Epoch: {epoch}\n\t Training: {train_metrics}\n\t Validation: {val_metrics}")
            
            # Save best model
            if val_metrics['loss'] < self.best_val_loss:
                self.best_val_loss = val_metrics['loss']
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': self.model.state_dict(),
                    'optimizer_state_dict': self.optimizer.state_dict(),
                    'val_loss': val_metrics['loss'],
                    'config': self.config.to_dict()
                }, self.best_model_path)
                print(f"  ✓ Saved best model (val_loss: {val_metrics['loss']:.4f}) epoch: {epoch}")
                self._epochs_no_improve = 0
            else:
                self._epochs_no_improve += 1

            if self.early_stop_patience and self._epochs_no_improve >= self.early_stop_patience:
                print("⏹️  Early stopping triggered")
                break
        
        print(f"\n✅ Training completed! Best val loss: {self.best_val_loss:.4f}")
        return self.best_val_loss
    def _train_epoch(self, epoch):
        """Training per una epoch"""
        self.model.train()
        
        total_loss = 0
        pos_loss = 0
        rot_loss = 0
        grip_loss = 0
        smooth_loss = 0
        for step, batch in enumerate(self.train_loader):
            obs = batch['observations'].to(self.device, non_blocking=True)
            target_actions = batch['actions'].to(self.device, non_blocking=True)
            self.optimizer.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(enabled=self.use_amp):
                pred_actions = self.model(obs)
                # Split components
                pos_pred, rot_pred, grip_pred = pred_actions[:,:3], pred_actions[:,3:6], pred_actions[:,6:7]
                pos_true, rot_true, grip_true = target_actions[:,:3], target_actions[:,3:6], target_actions[:,6:7]
                # Huber loss
                pos_l = F.huber_loss(pos_pred, pos_true, delta=1.0)
                rot_l = F.huber_loss(rot_pred, rot_true, delta=1.0)
                grip_l = F.huber_loss(grip_pred, grip_true, delta=1.0)
                # Smoothness regularization
                if step > 0:
                    smooth_l = F.mse_loss(pred_actions, prev_pred)
                else:
                    smooth_l = torch.tensor(0.0, device=self.device)
                prev_pred = pred_actions.detach()
                # Weighted sum
                loss = 2.0*pos_l + 1.5*rot_l + 1.0*grip_l + 0.1*smooth_l
            if self.use_amp:
                self.scaler.scale(loss).backward()
                if self.grad_clip:
                    self.scaler.unscale_(self.optimizer)
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.grad_clip)
                self.scaler.step(self.optimizer)
                self.scaler.update()
            else:
                loss.backward()
                if self.grad_clip:
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.grad_clip)
                self.optimizer.step()
            total_loss += loss.item()
            pos_loss += pos_l.item()
            rot_loss += rot_l.item()
            grip_loss += grip_l.item()
            smooth_loss += smooth_l.item()
            if self.scheduler is not None:
                self.scheduler.step()
        n_batches = len(self.train_loader)
        return {
            'loss': total_loss / n_batches,
            'pos_loss': pos_loss / n_batches,
            'rot_loss': rot_loss / n_batches,
            'grip_loss': grip_loss / n_batches,
            'smooth_loss': smooth_loss / n_batches
        }
    def _validate_epoch(self, epoch):
        self.model.eval()
        total_loss = 0
        pos_loss = 0
        rot_loss = 0
        grip_loss = 0
        smooth_loss = 0
        prev_pred = None
        with torch.no_grad():
            for batch in self.val_loader:
                obs = batch['observations'].to(self.device)
                target_actions = batch['actions'].to(self.device)
                pred_actions = self.model(obs)
                pos_pred, rot_pred, grip_pred = pred_actions[:,:3], pred_actions[:,3:6], pred_actions[:,6:7]
                pos_true, rot_true, grip_true = target_actions[:,:3], target_actions[:,3:6], target_actions[:,6:7]
                pos_l = F.huber_loss(pos_pred, pos_true, delta=1.0)
                rot_l = F.huber_loss(rot_pred, rot_true, delta=1.0)
                grip_l = F.huber_loss(grip_pred, grip_true, delta=1.0)
                # --- FIX: Only compute smooth_l if shapes match ---
                if prev_pred is not None and prev_pred.shape == pred_actions.shape:
                    smooth_l = F.mse_loss(pred_actions, prev_pred)
                else:
                    smooth_l = torch.tensor(0.0, device=self.device)
                prev_pred = pred_actions
                loss = 2.0*pos_l + 1.0*rot_l + 1.0*grip_l
                total_loss += loss.item()
                pos_loss += pos_l.item()
                rot_loss += rot_l.item()
                grip_loss += grip_l.item()
                smooth_loss += smooth_l.item()
        n_batches = len(self.val_loader)
        return {
            'loss': total_loss / n_batches,
            'pos_loss': pos_loss / n_batches,
            'rot_loss': rot_loss / n_batches,
            'grip_loss': grip_loss / n_batches,
            'smooth_loss': smooth_loss / n_batches
        }
    def _log_metrics(self, epoch, train_metrics, val_metrics):
        lr = self.optimizer.param_groups[0]['lr']
        print(f"Epoch {epoch}, train loss {train_metrics['loss']}, val loss {val_metrics['loss']}")
        print(f"  [pos] train: {train_metrics['pos_loss']:.4f} val: {val_metrics['pos_loss']:.4f}")
        print(f"  [rot] train: {train_metrics['rot_loss']:.4f} val: {val_metrics['rot_loss']:.4f}")
        print(f"  [grip] train: {train_metrics['grip_loss']:.4f} val: {val_metrics['grip_loss']:.4f}")
        print(f"  [smooth] train: {train_metrics['smooth_loss']:.4f} val: {val_metrics['smooth_loss']:.4f}")
        if self.use_wandb:
            wandb.log({
                'epoch': epoch,
                'train/loss': train_metrics['loss'],
                'train/pos_loss': train_metrics['pos_loss'],
                'train/rot_loss': train_metrics['rot_loss'],
                'train/grip_loss': train_metrics['grip_loss'],
                'train/smooth_loss': train_metrics['smooth_loss'],
                'val/loss': val_metrics['loss'],
                'val/pos_loss': val_metrics['pos_loss'],
                'val/rot_loss': val_metrics['rot_loss'],
                'val/grip_loss': val_metrics['grip_loss'],
                'val/smooth_loss': val_metrics['smooth_loss'],
                'lr': lr
            })

def build_dataloaders(
    train_dataset: Dataset,
    val_dataset: Dataset,
    batch_size: int,
    loader_kwargs: Dict[str, Any]
) -> Tuple[DataLoader, DataLoader]:
    """Crea data loader consistenti a partire da un template di kwargs."""

    kwargs = loader_kwargs.copy()
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
        **kwargs
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        drop_last=False,
        **kwargs
    )

    return train_loader, val_loader


def _set_dataset_augmentation(dataset, flag: bool):
    """Imposta augmentation temporaneamente e restituisce funzione di restore."""

    if not hasattr(dataset, 'augmentation'):
        return lambda: None

    original = dataset.augmentation
    dataset.augmentation = flag

    def restore():
        dataset.augmentation = original

    return restore


def optuna_random_search(
    train_dataset: Dataset,
    val_dataset: Dataset,
    loader_kwargs: Dict[str, Any],
    device: torch.device,
    quick_epochs: int = 10,
    search_space: Optional[HyperparameterSearchSpace] = None,
    n_trials: int = 50
) -> Tuple[TrainingConfig, List[Dict[str, Any]]]:
    """
    Randomized Optuna search (TPE-based) instead of grid-search.
    Runs a fixed number of trials (n_trials).
    """

    search_space = search_space or default_search_space()

    # --- NEW: TPE Sampler (recommended over random sampling) ---
    sampler = optuna.samplers.TPESampler(seed=42)

    study = optuna.create_study(
        direction='minimize',
        sampler=sampler
    )

    print(f"\n🔍 Starting Optuna random search")
    print(f"  Number of trials: {n_trials}")
    print(f"  Quick epochs: {quick_epochs}")

    def objective(trial: optuna.Trial) -> float:

        # --- Suggest values from the search space ---
        trial_config = TrainingConfig(
            lr=trial.suggest_categorical('lr', search_space.lr),
            hidden_dim=trial.suggest_categorical('hidden_dim', search_space.hidden_dim),
            num_recursions=trial.suggest_categorical('num_recursions', search_space.num_recursions),
            epochs=quick_epochs,
            batch_size=trial.suggest_categorical('batch_size', search_space.batch_size),
            weight_decay=trial.suggest_categorical('weight_decay', search_space.weight_decay),
            use_pretrained_encoder=trial.suggest_categorical('pretrained_encoder', search_space.pretrained_encoder),
            freeze_backbone=trial.suggest_categorical('freeze_backbone', search_space.freeze_backbone),
            augmentation=trial.suggest_categorical('augmentation', search_space.augmentation),
            dropout=trial.suggest_categorical('dropout', search_space.dropout),
            encoder_dropout=trial.suggest_categorical('dropout', search_space.dropout),
            grad_clip=1.0,
            save_path=f"optuna_trial_{trial.number}.pt"
        )

        # Build dataloaders based on this trial's batch size
        train_loader, val_loader = build_dataloaders(
            train_dataset,
            val_dataset,
            trial_config.batch_size,
            loader_kwargs
        )

        # Build model
        model = build_policy_from_config(trial_config)
        trainer = BehaviorCloningTrainer(
            model,
            train_loader,
            val_loader,
            trial_config,
            device,
            use_wandb=False
        )

        # Turn augmentation on/off safely
        restore_aug = _set_dataset_augmentation(train_dataset, trial_config.augmentation)

        try:
            val_loss = trainer.train()
        finally:
            restore_aug()

        # Store config inside trial for later retrieval
        trial.set_user_attr('config', trial_config.to_dict())

        return val_loss

    # --- NEW: Run fixed-size random search ---
    study.optimize(objective, n_trials=n_trials)

    # Save trial history (full information)
    history = []
    for trial in study.trials:
        history.append({
            'trial_number': trial.number,
            'state': str(trial.state),
            'value': trial.value,
            'params': trial.params,                # all parameters sampled
            'user_attrs': trial.user_attrs,        # user metadata (our config)
            'distributions': {
                k: str(v) for k, v in trial.distributions.items()
            },                                      # searchable distributions
            'system_attrs': trial.system_attrs,    # optuna internal info
            'datetime_start': str(trial.datetime_start),
            'datetime_complete': str(trial.datetime_complete),
        })


    # Best configuration
    best_config = TrainingConfig(**study.best_trial.user_attrs['config'])

    return best_config, history


In [16]:
def train_final_model(
    base_config: TrainingConfig,
    train_dataset: Dataset,
    val_dataset: Dataset,
    loader_kwargs: Dict[str, Any],
    device,
    final_epochs: Optional[int] = None,
    use_wandb: bool = False
) -> Tuple[nn.Module, float]:
    """Esegue il training finale a partire dalla configurazione scelta."""

    final_epochs = final_epochs or base_config.epochs
    final_config = replace(
        base_config,
        epochs=final_epochs,
        save_path='final_model.pt',
        early_stop_patience=base_config.early_stop_patience or 10,
        sched_T0=base_config.sched_T0 or max(1, final_epochs // 5)
    )

    print(f"\n{'='*60}")
    print("🎯 FINAL TRAINING")
    print(f"Config: {final_config.label()}")
    print(f"{'='*60}")

    if use_wandb:
        wandb.init(
            project="trm-robotics",
            name=f"final_{final_config.label()}",
            config=final_config.to_dict()
        )

    train_loader, val_loader = build_dataloaders(
        train_dataset,
        val_dataset,
        final_config.batch_size,
        loader_kwargs
    )

    model = build_policy_from_config(final_config)
    trainer = BehaviorCloningTrainer(
        model,
        train_loader,
        val_loader,
        final_config,
        device,
        use_wandb=use_wandb
    )

    restore_aug = _set_dataset_augmentation(train_dataset, final_config.augmentation)
    try:
        final_val_loss = trainer.train()
    finally:
        restore_aug()

    if os.path.exists(final_config.save_path):
        checkpoint = torch.load(final_config.save_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])

    if use_wandb:
        wandb.finish()

    print(f"\n✅ Final model trained! Val loss: {final_val_loss:.4f}")

    return model, final_val_loss

class PolicyEvaluator:
    """
    Valutatore per policy robotiche in simulazione LIBERO
    """
    
    def __init__(
        self,
        policy: nn.Module,
        device: torch.device,
        action_stats: Dict = None
    ):
        self.policy = policy.to(device)
        self.policy.eval()
        self.device = device
        self.action_stats = action_stats
    
    def evaluate_on_task(
        self,
        env,
        init_states,
        num_episodes=50,
        max_steps=500,
        record_video=False,
        video_path=None
    ):
        """
        Valuta policy su un singolo task
        
        Args:
            env: environment LIBERO
            init_states: stati iniziali per il task
            num_episodes: numero di episodi da valutare
            max_steps: massimo numero di step per episodio
            record_video: se True, registra video
            video_path: path per salvare video
        
        Returns:
            results: dizionario con metriche
        """
        successes = []
        episode_lengths = []
        frames_buffer = [] if record_video else None
        
        for ep in range(num_episodes):
            # Reset environment
            env.reset()
            env.set_init_state(init_states[ep % len(init_states)])
            
            obs = env.get_observation()
            done = False
            step = 0
            
            episode_frames = [] if record_video and ep < 5 else None
            
            while not done and step < max_steps:
                # Cattura frame
                if episode_frames is not None:
                    frame = env.render(mode='rgb_array')
                    episode_frames.append(frame)
                
                # Preprocessing osservazione
                obs_tensor = self._preprocess_obs(obs)
                
                # Predici azione
                with torch.no_grad():
                    action = self.policy(obs_tensor)
                    action = action.cpu().numpy().squeeze()
                    
                    # Denormalizza azione se necessario
                    if self.action_stats is not None:
                        action = action * self.action_stats['std'] + self.action_stats['mean']
                
                # Step environment
                obs, reward, done, info = env.step(action)
                step += 1
            
            # Registra risultati
            success = info.get('success', False)
            successes.append(success)
            episode_lengths.append(step)
            
            if episode_frames is not None:
                frames_buffer.append(episode_frames)
        
        # Salva video se richiesto
        if record_video and frames_buffer and video_path:
            self._save_videos(frames_buffer, video_path)
        
        # Calcola metriche
        results = {
            'success_rate': np.mean(successes),
            'avg_episode_length': np.mean(episode_lengths),
            'std_episode_length': np.std(episode_lengths),
            'num_episodes': num_episodes
        }
        
        return results
    
    def _preprocess_obs(self, obs):
        """Preprocessing osservazione per policy"""
        # Estrai immagine RGB
        img = obs['agentview_rgb']
        
        # Normalizza
        img = img.astype(np.float32) / 255.0
        
        # Converti a tensor e aggiungi batch dimension
        img = np.transpose(img, (2, 0, 1))  # HWC -> CHW
        img_tensor = torch.from_numpy(img).unsqueeze(0).to(self.device)
        
        return img_tensor
    
    def _save_videos(self, frames_buffer, video_path):
        """Salva video degli episodi"""
        os.makedirs(os.path.dirname(video_path), exist_ok=True)
        
        for i, frames in enumerate(frames_buffer):
            path = video_path.replace('.mp4', f'_ep{i}.mp4')
            
            # Usa OpenCV per salvare video
            height, width, _ = frames[0].shape
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(path, fourcc, 30.0, (width, height))
            
            for frame in frames:
                frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
                out.write(frame_bgr)
            
            out.release()
            print(f"  Video salvato: {path}")


def generate_evaluation_report(results_dict, output_path):
    """
    Genera report HTML con risultati di valutazione
    
    Args:
        results_dict: dizionario {task_name: results}
        output_path: path per salvare report
    """
    
    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>TRM Robotics - Evaluation Report</title>
        <style>
            body {{ font-family: Arial, sans-serif; margin: 40px; }}
            h1 {{ color: #333; }}
            table {{ border-collapse: collapse; width: 100%; margin-top: 20px; }}
            th, td {{ border: 1px solid #ddd; padding: 12px; text-align: left; }}
            th {{ background-color: #4CAF50; color: white; }}
            tr:nth-child(even) {{ background-color: #f2f2f2; }}
            .success {{ color: green; font-weight: bold; }}
            .fail {{ color: red; }}
        </style>
    </head>
    <body>
        <h1>🤖 TRM Robotics Evaluation Report</h1>
        <p><strong>Generated:</strong> {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}</p>
        
        <h2>Results Summary</h2>
        <table>
            <tr>
                <th>Task Name</th>
                <th>Success Rate</th>
                <th>Avg Episode Length</th>
                <th>Std Episode Length</th>
                <th>Num Episodes</th>
            </tr>
    """
    
    # Aggiungi righe per ogni task
    overall_success = []
    
    for task_name, results in results_dict.items():
        success_rate = results['success_rate']
        overall_success.append(success_rate)
        
        success_class = 'success' if success_rate >= 0.5 else 'fail'
        
        html_content += f"""
            <tr>
                <td>{task_name}</td>
                <td class="{success_class}">{success_rate:.2%}</td>
                <td>{results['avg_episode_length']:.1f}</td>
                <td>{results['std_episode_length']:.1f}</td>
                <td>{results['num_episodes']}</td>
            </tr>
        """
    
    # Overall statistics
    mean_success = np.mean(overall_success)
    success_class = 'success' if mean_success >= 0.5 else 'fail'
    
    html_content += f"""
            <tr style="font-weight: bold; background-color: #e0e0e0;">
                <td>OVERALL</td>
                <td class="{success_class}">{mean_success:.2%}</td>
                <td colspan="3">-</td>
            </tr>
        </table>
    </body>
    </html>
    """
    
    # Salva report
    with open(output_path, 'w') as f:
        f.write(html_content)
    
    print(f"\n✓ Report salvato: {output_path}")

In [20]:
def main_pipeline(
    data_path: str = '/kaggle/working/dataset/libero_spatial',
    work_dir: str = '/kaggle/working/trm_robotics',
    quick_search: bool = True,
    train_final: bool = True,
    use_custom_final_config: bool = False,
    evaluate: bool = True,
    use_wandb: bool = False
):
    """
    Pipeline completa per il progetto TRM Robotics
    
    Args:
        data_path: path ai dati LIBERO
        work_dir: directory di lavoro
        quick_search: se True, esegue hyperparameter search
        train_final: se True, esegue training finale
        evaluate: se True, esegue valutazione in simulazione
        use_wandb: se True, usa WandB per logging
    """
    
    print(f"""
    {'='*80}
    🤖 TinyRecursiveModels per Controllo Robotico
    {'='*80}
    
    Obiettivi:
    1. Adattare architettura TRM per robotica
    2. Training con Behavior Cloning su LIBERO
    3. Valutazione in simulazione con metriche quantitative e qualitative
    
    """)
    
    # Setup directories
    os.makedirs(work_dir, exist_ok=True)
    os.chdir(work_dir)
    
    # Device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"✓ Using device: {device}\n")
    
    # ========== STEP 1: Caricamento Dataset ==========
    print(f"\n{'='*80}")
    print("STEP 1: Caricamento Dataset")
    print(f"{'='*80}")
    
    # Trova file HDF5
    data_path = Path(data_path)
    hdf5_files = list(data_path.glob('**/*.hdf5'))
    
    if not hdf5_files:
        print(f"❌ Nessun file HDF5 trovato in {data_path}")
        print("Assicurati di aver scaricato il dataset LIBERO!")
        return
    
    print(f"✓ Trovati {len(hdf5_files)} file HDF5 (task)")
    
    # Demo-level split: usa TUTTI i file per entrambi i dataset
    # ma dividi le demo all'interno di ciascun file (80% train, 20% val)
    demo_split_ratio = 0.8
    print(f"\n📊 Demo-level split: {demo_split_ratio:.0%} train / {1-demo_split_ratio:.0%} val per ogni task")
    print(f"   Tutti i {len(hdf5_files)} task presenti in entrambi train e val")
    
    # Crea datasets con demo-level split
    print("\nCreating TRAIN dataset...")
    train_dataset = LIBERODataset(
        hdf5_files,  # Usa TUTTI i file
        sequence_length=1,
        image_size=(128, 128),
        augmentation=False,
        max_demos_per_task=50,  # Limita per velocità
        demo_split_ratio=demo_split_ratio,
        is_train=True  # Prime 80% demo per task
    )
    
    # Usa le stesse statistiche del training set per la validation
    train_action_stats = train_dataset.action_stats
    
    print("\nCreating VAL dataset...")
    val_dataset = LIBERODataset(
        hdf5_files,  # Usa TUTTI i file
        sequence_length=1,
        image_size=(128, 128),
        augmentation=False,
        max_demos_per_task=50,  # Stesso limite per coerenza
        demo_split_ratio=demo_split_ratio,
        is_train=False,  # Ultime 20% demo per task
        action_stats=train_action_stats  # Usa statistiche del training
    )
    
    # Data loader template (verranno istanziati on-demand)
    num_workers = min(4, os.cpu_count() or 1)
    use_cuda = torch.cuda.is_available()

    loader_common = {
        'num_workers': num_workers,
        'pin_memory': use_cuda,
        'persistent_workers': num_workers > 0
    }
    if num_workers > 0:
        loader_common['prefetch_factor'] = 2

    print(f"\n✓ Dataset creati con demo-level split")
    print(f"  Train samples: {len(train_dataset)}")
    print(f"  Val samples: {len(val_dataset)}")
    
    # Salva action stats per denormalizzazione
    action_stats = train_dataset.action_stats
    with open('action_stats.json', 'w') as f:
        json.dump({
            'mean': action_stats['mean'].tolist(),
            'std': action_stats['std'].tolist()
        }, f)
    
    # ========== STEP 2: Hyperparameter Search ==========
    best_config = None
    
    if quick_search:
        print(f"\n{'='*80}")
        print("STEP 2: Hyperparameter Search")
        print(f"{'='*80}")
        
        best_config, all_results = optuna_random_search(
            train_dataset,
            val_dataset,
            loader_common,
            device,
            quick_epochs=10
        )
        
        # Salva risultati
        with open('hyperparam_results.json', 'w') as f:
            json.dump(all_results, f, indent=2)
        with open('best_hyperparams.json', 'w') as f:
            json.dump(best_config.to_dict(), f, indent=2)
    
    # ========== STEP 3: Training Finale ==========
    trained_model = None
    
    if train_final:
        print(f"\n{'='*80}")
        print("STEP 3: Training Finale")
        print(f"{'='*80}")
        
        if best_config is None or use_custom_final_config:
            best_config = TrainingConfig(
                lr=3e-4,
                hidden_dim=256,
                num_recursions=8,
                batch_size=32,
                epochs=100,
                weight_decay=1e-4,
                dropout=0.3,
                encoder_dropout=0.1,
                use_pretrained_encoder=True,
                freeze_backbone=False,
                augmentation=False
            )
            print("⚠️  Usando configurazione custom di default")

        trained_model, final_val_loss = train_final_model(
            best_config,
            train_dataset,
            val_dataset,
            loader_common,
            device,
            final_epochs=100,
            use_wandb=use_wandb
        )
    
    # ========== STEP 4: Valutazione ==========
    if evaluate:
        print(f"\n{'='*80}")
        print("STEP 4: Valutazione in Simulazione")
        print(f"{'='*80}")
        
        # Carica modello se non già trainato
        if trained_model is None:
            print("Caricando modello salvato...")
            checkpoint = torch.load('final_model.pt')
            config_dict = checkpoint['config'] if 'config' in checkpoint else {}
            # Usa build_policy_from_config per garantire coerenza architetturale
            trained_model = build_policy_from_config(
                TrainingConfig(**config_dict),
                obs_shape=(3, 128, 128)
            )
            trained_model.load_state_dict(checkpoint['model_state_dict'])
            trained_model = trained_model.to(device)
        
        # Crea evaluator
        evaluator = PolicyEvaluator(trained_model, device, action_stats)
        
        # Nota: La valutazione in simulazione richiede setup di LIBERO environment
        # Questo è un placeholder per la struttura
        print("\n⚠️  La valutazione in simulazione richiede setup completo di LIBERO")
        print("Vedi sezione 'Valutazione in Simulazione' per implementazione completa")
        
        # Pseudocodice per valutazione:
        """
        from libero.libero import benchmark
        
        results_dict = {}
        
        for task_id in range(num_tasks):
            env, init_states = create_eval_env(task_suite, task_id)
            
            results = evaluator.evaluate_on_task(
                env,
                init_states,
                num_episodes=50,
                record_video=True,
                video_path=f'videos/task_{task_id}.mp4'
            )
            
            results_dict[f'task_{task_id}'] = results
            env.close()
        
        # Genera report
        generate_evaluation_report(results_dict, 'evaluation_report.html')
        """
    
    print(f"\n{'='*80}")
    print("✅ Pipeline completata!")
    print(f"{'='*80}")
    print(f"\nFile generati in: {work_dir}")
    print("  - final_model.pt: modello allenato")
    print("  - action_stats.json: statistiche azioni")
    print("  - hyperparam_results.json: risultati hyperparameter search")

In [ ]:
# if __name__ == '__main__':
#     # Configura parametri
#     CONFIG = {
#         'data_path': '/kaggle/working/dataset/libero_spatial',
#         'work_dir': '/kaggle/working/trm_robotics',
#         'quick_search': True,
#         'train_final': True,
#         'use_custom_final_config': True,
#         'evaluate': True,  # Richiede setup LIBERO completo
#         'use_wandb': False  # Imposta True per logging WandB
#     }
    
#     # Esegui pipeline
#     main_pipeline(**CONFIG)

In [ ]:
"""Setup model with Optuna best params (trial 26), optionally rerun Optuna, then validate and train."""
import torch
from dataclasses import replace

data_path =  '/kaggle/working/LIBERO/libero/datasets/libero_spatial'
work_dir = '/kaggle/working/trm_robotics'

# Optuna best parameters from your message
best_params = {
    'lr': 1e-6,
    'hidden_dim': 128,
    'num_recursions': 12,
    'batch_size': 256,
    'weight_decay': 1.0,
    'use_pretrained_encoder': True,
    'freeze_backbone': False,
    'augmentation': True,
    'dropout': 0.3,
}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✓ Using device: {device}\n")

# ========== STEP 1: Caricamento Dataset ==========
print(f"\n{'='*80}")
print("STEP 1: Caricamento Dataset")
print(f"{'='*80}")

# Trova file HDF5
data_path = Path(data_path)
hdf5_files = list(data_path.glob('**/*.hdf5'))

if not hdf5_files:
    print(f"❌ Nessun file HDF5 trovato in {data_path}")
    print("Assicurati di aver scaricato il dataset LIBERO!")

print(f"✓ Trovati {len(hdf5_files)} file HDF5 (task)")

print(f"Using only the first task: {hdf5_files[0].name}")

# Demo-level split: usa TUTTI i file per entrambi i dataset
# ma dividi le demo all'interno di ciascun file (80% train, 20% val)
demo_split_ratio = 0.8
print(f"\n📊 Demo-level split: {demo_split_ratio:.0%} train / {1-demo_split_ratio:.0%} val per ogni task")
print(f"   Tutti i {len(hdf5_files)} task presenti in entrambi train e val")

# Crea datasets con demo-level split
print("\nCreating TRAIN dataset...")
train_dataset = LIBERODataset(
    hdf5_files,  # Usa TUTTI i file
    sequence_length=1,
    image_size=(128, 128),
    augmentation=False,
    max_demos_per_task=50,  # Limita per velocità
    demo_split_ratio=demo_split_ratio,
    is_train=True  # Prime 80% demo per task
)

# Usa le stesse statistiche del training set per la validation
train_action_stats = train_dataset.action_stats

print("\nCreating VAL dataset...")
val_dataset = LIBERODataset(
    hdf5_files,  # Usa TUTTI i file
    sequence_length=1,
    image_size=(128, 128),
    augmentation=False,
    max_demos_per_task=50,  # Stesso limite per coerenza
    demo_split_ratio=demo_split_ratio,
    is_train=False,  # Ultime 20% demo per task
    action_stats=train_action_stats  # Usa statistiche del training
)

# Data loader template (verranno istanziati on-demand)
num_workers = min(4, os.cpu_count() or 1)
use_cuda = torch.cuda.is_available()

loader_common = {
    'num_workers': num_workers,
    'pin_memory': use_cuda,
    'persistent_workers': num_workers > 0
}
if num_workers > 0:
    loader_common['prefetch_factor'] = 2

print(f"\n✓ Dataset creati con demo-level split")
print(f"  Train samples: {len(train_dataset)}")
print(f"  Val samples: {len(val_dataset)}")

# Salva action stats per denormalizzazione
action_stats = train_dataset.action_stats
with open('action_stats.json', 'w') as f:
    json.dump({
        'mean': action_stats['mean'].tolist(),
        'std': action_stats['std'].tolist()
    }, f)

# Build TrainingConfig or use fresh Optuna search
run_optuna_search = True
optuna_trials = 5
optuna_epochs = 10

if run_optuna_search:
    print("\n=== OPTUNA SEARCH (best_params-style) ===")
    best_config, optuna_history = optuna_random_search(
        train_dataset=train_dataset,
        val_dataset=val_dataset,
        loader_kwargs=loader_common,
        device=device,
        quick_epochs=optuna_epochs,
        n_trials=optuna_trials
    )
    with open('hyperparam_results.json', 'w') as f:
        json.dump(optuna_history, f, indent=2)
    with open('best_hyperparams.json', 'w') as f:
        json.dump(best_config.to_dict(), f, indent=2)
    config = replace(
        best_config,
        epochs=50,
        save_path='best_trial_26.pt'
    )
    print(f"Using Optuna best config: {config.label()}")
else:
    config = TrainingConfig(
        lr=best_params['lr'],
        hidden_dim=best_params['hidden_dim'],
        num_recursions=best_params['num_recursions'],
        epochs=50,
        batch_size=best_params['batch_size'],
        weight_decay=best_params['weight_decay'],
        use_pretrained_encoder=best_params['use_pretrained_encoder'],
        freeze_backbone=best_params['freeze_backbone'],
        augmentation=best_params['augmentation'],
        dropout=best_params['dropout'],
        encoder_dropout=best_params['dropout'],
        save_path='best_trial_26.pt'
    )

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Retrieve datasets and loader kwargs from notebook namespace (created earlier)
train_dataset = globals().get('train_dataset')
val_dataset = globals().get('val_dataset')
loader_common = globals().get('loader_common', {'num_workers': 0, 'pin_memory': False, 'persistent_workers': False})

if train_dataset is None or val_dataset is None:
    raise RuntimeError("train_dataset or val_dataset not found. Run the dataset preparation cells before this cell.")

# Build dataloaders, model and trainer
train_loader, val_loader = build_dataloaders(train_dataset, val_dataset, config.batch_size, loader_common)
model = build_policy_from_config(config, obs_shape=(3, 128, 128)).to(device)
trainer = BehaviorCloningTrainer(model, train_loader, val_loader, config, device, use_wandb=False)

# Initial validation (before training) and training
print('Running initial validation...')
val_metrics = trainer._validate_epoch(0)
print(f"Initial validation loss: {val_metrics['loss']:.4f}")

print('Starting training...')
best_val = trainer.train()
print(f"Training finished. Best validation loss saved: {best_val:.4f}")
print(f"Checkpoint saved to: {config.save_path}")

✓ Using device: cuda


STEP 1: Caricamento Dataset
✓ Trovati 4 file HDF5 (task)
Using only the first task: pick_up_the_black_bowl_on_the_wooden_cabinet_and_place_it_on_the_plate_demo.hdf5

📊 Demo-level split: 80% train / 20% val per ogni task
   Tutti i 4 task presenti in entrambi train e val

Creating TRAIN dataset...
Loading 4 HDF5 files for TRAIN (demo split: 80%)...
✅ Loaded 160 demonstrations for TRAIN
📊 Action statistics computed from TRAIN set:
   Mean:        [ 0.153  0.131 -0.145 -0.005 -0.012 -0.023  0.086]
   Std (raw):   [0.449 0.349 0.533 0.041 0.075 0.057 0.996]
   Std (clipped to >=0.1): [0.449 0.349 0.533 0.1   0.1   0.1   0.996]
📦 Generated 19278 transitions for TRAIN

Creating VAL dataset...
Loading 4 HDF5 files for VAL (demo split: 80%)...
✅ Loaded 40 demonstrations for VAL
📊 Using provided action statistics
📦 Generated 4778 transitions for VAL

✓ Dataset creati con demo-level split
  Train samples: 19278
  Val samples: 4778
Running initial validation...


/tmp/ipykernel_47/1161071716.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.use_amp)


Initial validation loss: 2.6588
Starting training...


/tmp/ipykernel_47/1161071716.py:102: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.use_amp):


Epoch 0, train loss 3.455023628870646, val loss 2.762466920049567
  [pos] train: 0.8006 val: 0.7363
  [rot] train: 0.5275 val: 0.4652
  [grip] train: 0.9257 val: 0.8247
  [smooth] train: 1.3692 val: 0.5287
  ✓ Saved best model (val_loss: 2.7625) epoch: 0


KeyboardInterrupt: 

## weird trick 
Why this works

The sys.modules["matplotlib"] = MagicMock() lines tell Python: "If anyone asks for matplotlib, just give them this fake empty object." When libero tries to import matplotlib.cm, it gets the fake object instead of trying to load the actual file from disk. Since libero likely only uses matplotlib for color maps (which aren't critical for simple RGB evaluation), the code will proceed without crashing

In [ ]:
import sys
import numpy as np
from unittest.mock import MagicMock

# --- 1. MONKEY PATCH (CRITICAL FIX) ---
# We force Python to use a fake 'matplotlib' so it doesn't try to load the broken real one.
# This must be done BEFORE importing robosuite or libero.
mock_mpl = MagicMock()
sys.modules["matplotlib"] = mock_mpl
sys.modules["matplotlib.pyplot"] = mock_mpl
sys.modules["matplotlib.cm"] = mock_mpl
sys.modules["matplotlib.colors"] = mock_mpl
sys.modules["matplotlib.transforms"] = mock_mpl
sys.modules["matplotlib.ticker"] = mock_mpl
sys.modules["matplotlib._path"] = mock_mpl
# --------------------------------------

import torch
import json
import os
from pathlib import Path
from typing import Any, Dict, List, Callable
from dataclasses import dataclass

# --- 2. SETUP PATHS ---
LIBERO_REPO_ROOT = Path('/kaggle/working/LIBERO')
if LIBERO_REPO_ROOT.exists() and str(LIBERO_REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(LIBERO_REPO_ROOT))

# Fix for Numba
try:
    from robosuite.utils.numba import jit_decorator
except Exception:
    pass

# Now these imports will work because matplotlib is mocked
from libero.libero import get_libero_path
from libero.libero.benchmark import get_benchmark
from libero.libero.envs import OffScreenRenderEnv
from libero.libero.utils.time_utils import Timer
from libero.libero.utils.video_utils import VideoWriter

# --- 3. SEQUENTIAL VECTOR ENV (Crash-Proof Class) ---
class SequentialVectorEnv:
    def __init__(self, env_fns: List[Callable]):
        self.envs = [fn() for fn in env_fns]

    def step(self, actions):
        results = [env.step(a) for env, a in zip(self.envs, actions)]
        obs_list, rews_list, dones_list, infos_list = zip(*results)
        return list(obs_list), np.array(rews_list), np.array(dones_list), list(infos_list)

    def reset(self):
        return [env.reset() for env in self.envs]

    def seed(self, seed):
        for i, env in enumerate(self.envs):
            if hasattr(env, 'seed'):
                env.seed(seed + i)

    def set_init_state(self, states):
        return [env.set_init_state(s) for env, s in zip(self.envs, states)]

    def close(self):
        for env in self.envs:
            env.close()

def _stack_vector_obs(obs: Any) -> Dict[str, np.ndarray]:
    if isinstance(obs, list):
        keys = obs[0].keys()
        return {k: np.stack([o[k] for o in obs], axis=0) for k in keys}
    return obs

def _select_camera_key(obs_batch: Dict[str, np.ndarray]) -> str:
    for key in ('agentview_rgb', 'agentview_image', 'robot0_agentview_image'):
        if key in obs_batch: return key
    return list(obs_batch.keys())[0]

def _prepare_policy_input(images: np.ndarray, device: torch.device) -> torch.Tensor:
    imgs = torch.from_numpy(images).to(device=device, dtype=torch.float32) / 255.0
    return imgs.permute(0, 3, 1, 2).contiguous()

# --- 5. EVALUATION FUNCTION ---
def evaluate_best_trial_26(
    checkpoint_path: str = 'best_trial_26.pt',
    action_stats_path: str = 'action_stats.json',
    benchmark: str = 'libero_spatial',
    task_id: int = 2,  # Fixed to 0 to match training task
    env_num: int = 1,
    max_steps: int = 300,
    seed: int = 42,
    save_videos: bool = True,
    video_dir: str = 'evaluation_videos',
    camera_height: int = 128,
    camera_width: int = 128,
    video_skip: int = 1
):
    print(f"Starting evaluation on {benchmark} Task {task_id} (same as training)...")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Load Model
    ckpt = torch.load(checkpoint_path, map_location=device, weights_only=False)
    cfg_dict = ckpt.get('config', {}) or {}
    base_cfg = TrainingConfig()
    merged_cfg = base_cfg.to_dict()
    merged_cfg.update(cfg_dict)
    cfg = TrainingConfig(**merged_cfg)

    policy = build_policy_from_config(cfg, obs_shape=(3, camera_height, camera_width)).to(device)
    policy.load_state_dict(ckpt['model_state_dict'])
    policy.eval()

    # Load Stats
    stats = json.load(open(action_stats_path))
    action_mean = torch.tensor(stats['mean'], device=device).unsqueeze(0)
    action_std = torch.tensor(stats['std'], device=device).unsqueeze(0)

    # Setup Env
    benchmark_map = {'libero_10': 'LIBERO_10', 'libero_spatial': 'LIBERO_SPATIAL'}
    suite = get_benchmark(benchmark_map.get(benchmark, benchmark))(0)
    task = suite.get_task(task_id)
    
    print(f"Evaluating task: {task.name} (task_id={task_id})")
    
    env_args = {
        'bddl_file_name': str(Path(get_libero_path('bddl_files')) / task.problem_folder / task.bddl_file),
        'camera_heights': camera_height,
        'camera_widths': camera_width
    }
    
    # Initialize SequentialVectorEnv (Mocked matplotlib allows this to load now)
    env = SequentialVectorEnv([lambda: OffScreenRenderEnv(**env_args) for _ in range(env_num)])
    
    try:
        init_states = torch.load(str(Path(get_libero_path('init_states')) / task.problem_folder / task.init_states_file), map_location='cpu', weights_only=False)
        env.reset()
        env.seed(seed)
        env.set_init_state(init_states[0:env_num])
        
        # Warmup
        for _ in range(2): env.step(np.zeros((env_num, action_mean.shape[-1])))

        with VideoWriter(video_dir, save_videos) as video_writer:
            obs = env.reset()
            obs = env.set_init_state(init_states[0:env_num])
            dones = [False] * env_num
            
            for step in range(max_steps):
                obs_batch = _stack_vector_obs(obs)
                cam_key = _select_camera_key(obs_batch)
                
                # Inference
                p_in = _prepare_policy_input(obs_batch[cam_key], device)
                actions = policy(p_in) * action_std + action_mean
                actions_np = actions.detach().cpu().numpy()

                obs, reward, done, info = env.step(actions_np)

                # --- PRINT DEBUG INFO ---
                print(f"\n[STEP {step}]")
                for env_id, info in enumerate(info_list):
                    if "evaluation_info" in info:
                        eval_info = info["evaluation_info"]
                        print(f"\nEnv {env_id} → BDDL evaluation:")
                        
                        # Print full dict
                        print(json.dumps(eval_info, indent=4))

                        # Highlight failing conditions
                        failed = [k for k, v in eval_info.items() if v is False]
                        passed  = [k for k, v in eval_info.items() if v is True]

                        print("\n  ✔ Passed conditions:")
                        for k in passed:
                            print(f"     - {k}")

                        print("\n  ❌ Failed conditions:")
                        for k in failed:
                            print(f"     - {k}")

                        # Success indicator
                        if done_batch[env_id]:
                            print("\n  🎉 DONE FLAG = TRUE → Task satisfied according to BDDL")
                        else:
                            print("\n  ⚠ DONE FLAG = FALSE → Task NOT satisfied according to BDDL")

                    else:
                        print("No evaluation_info present in info dict.")

                # Save video
                if save_videos and step % video_skip == 0:
                    video_writer.append_vector_obs(obs_batch, dones, camera_name=cam_key)

                # Break if all done
                if all(done_batch):
                    print("\nAll environments report done=True. Stopping evaluation.")
                    break

                
        print("Success! Videos saved.")
    finally:
        env.close()

# Esempio di chiamata:
evaluate_trm_libero(
    policy=policy,
    benchmark="libero_spatial",
    task_id=0,
    env_num=1,
    max_steps=300,
    seed=100,
    save_videos=True,
    video_dir="evaluation_videos",
    camera_height=128,
    camera_width=128,
    video_skip=1
)

In [ ]:
def evaluate_full_suite(
    checkpoint_path: str = 'best_trial_26.pt',
    action_stats_path: str = 'action_stats.json',
    benchmark: str = 'libero_spatial',
    env_num: int = 1,
    max_steps: int = 300,
    seed: int = 42,
    save_videos: bool = True,
    video_dir: str = 'evaluation_videos',
    camera_height: int = 128,
    camera_width: int = 128,
    video_skip: int = 1
):
    print(f"--- EVALUATING FULL SUITE: {benchmark} ---")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # 1. Load Model & Stats
    print("Loading model...")
    repo_root = Path('/kaggle/working/LIBERO')
    if repo_root.exists():
        if str(repo_root) not in sys.path:
            sys.path.insert(0, str(repo_root))
    from libero.libero.benchmark import get_benchmark as _load_benchmark
    ckpt = torch.load(checkpoint_path, map_location=device, weights_only=False)
    cfg_dict = ckpt.get('config', {}) or {}
    base_cfg = TrainingConfig()
    merged_cfg = base_cfg.to_dict()
    merged_cfg.update(cfg_dict)
    cfg = TrainingConfig(**merged_cfg)

    policy = build_policy_from_config(cfg, obs_shape=(3, camera_height, camera_width)).to(device)
    policy.load_state_dict(ckpt['model_state_dict'])
    policy.eval()

    stats = json.load(open(action_stats_path))
    action_mean = torch.tensor(stats['mean'], device=device).unsqueeze(0)
    action_std = torch.tensor(stats['std'], device=device).unsqueeze(0)

    # 2. Get Benchmark Info
    benchmark_map = {'libero_10': 'LIBERO_10', 'libero_spatial': 'LIBERO_SPATIAL'}
    suite_name = benchmark_map.get(benchmark, benchmark)
    suite = _load_benchmark(suite_name)(0)
    num_tasks = suite.n_tasks
    print(f"Found {num_tasks} tasks in benchmark '{suite_name}'.")
    
    total_success = 0.0
    task_results = {}
    
    # 3. LOOP THROUGH ALL TASKS
    for task_id in range(num_tasks):
        task = suite.get_task(task_id)
        print(f"\n>>> Starting Task {task_id}: {task.name}")
        
        env_args = {
            'bddl_file_name': str(Path(get_libero_path('bddl_files')) / task.problem_folder / task.bddl_file),
            'camera_heights': camera_height,
            'camera_widths': camera_width
        }

        # Initialize Env
        env = SequentialVectorEnv([lambda: OffScreenRenderEnv(**env_args) for _ in range(env_num)])
        
        try:
            # Load Init States
            init_states_path = Path(get_libero_path('init_states')) / task.problem_folder / task.init_states_file
            init_states = torch.load(str(init_states_path), map_location='cpu', weights_only=False)
            
            # Reset & Warmup
            env.reset()
            env.seed(seed)
            indices = np.arange(env_num) % init_states.shape[0]
            env.set_init_state(init_states[indices])
            
            for _ in range(5): 
                env.step(np.zeros((env_num, action_mean.shape[-1])))

            # --- FIX: Create a unique subfolder for each task ---
            # This prevents overwrites without breaking the camera key lookup
            task_video_dir = os.path.join(video_dir, f"task_{task_id}")
            os.makedirs(task_video_dir, exist_ok=True)
            
            with VideoWriter(task_video_dir, save_videos) as video_writer:
                obs = env.reset()
                obs = env.set_init_state(init_states[indices])
                dones = [False] * env_num
                successes = [False] * env_num
                
                for step in range(max_steps):
                    obs_batch = _stack_vector_obs(obs)
                    cam_key = _select_camera_key(obs_batch)
                    
                    # Inference
                    p_in = _prepare_policy_input(obs_batch[cam_key], device)
                    actions = policy(p_in) * action_std + action_mean
                    actions_np = actions.detach().cpu().numpy()
                    
                    # Step
                    obs, _, done_batch, info = env.step(actions_np)
                    
                    # Track Success
                    for i in range(env_num):
                        if info[i].get('success', False):
                            successes[i] = True
                    
                    # Save Video
                    if save_videos and step % video_skip == 0:
                        obs_list = [{key: obs_batch[key][i] for key in obs_batch} for i in range(env_num)]
                        video_writer.append_vector_obs(obs_list, dones, camera_name=cam_key)
                    
                
            task_success_rate = np.mean(successes)
            total_success += task_success_rate
            status = "✅ SUCCESS" if task_success_rate > 0 else "❌ FAILURE"
            print(f"    [Task {task_id}] Finished. Success Rate: {task_success_rate:.2f} - {status}")
            task_results[f"task_{task_id}"] = {
                "name": task.name,
                "success_rate": float(task_success_rate)
            }

        finally:
            env.close()

    # 4. Final Summary
    avg_success = total_success / num_tasks
    print(f"\n==========================================")
    print(f"       FINAL EVALUATION REPORT            ")
    print(f"==========================================")
    print(f"Benchmark: {benchmark}")
    print(f"Total Tasks: {num_tasks}")
    print(f"Average Success Rate: {avg_success:.2f}")
    print(f"==========================================")

    return {
        "per_task": task_results,
        "average_success_rate": float(avg_success)
    }

--- EVALUATING FULL SUITE: libero_spatial ---
Loading model...
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Found 10 tasks in benchmark 'LIBERO_SPATIAL'.

>>> Starting Task 0: pick_up_the_black_bowl_between_the_plate_and_the_ramekin_and_place_it_on_the_plate
Saved videos to evaluation_videos/task_0.
    [Task 0] Finished. Success Rate: 0.00 - ❌ FAILURE

>>> Starting Task 1: pick_up_the_black_bowl_next_to_the_ramekin_and_place_it_on_the_plate
Saved videos to evaluation_videos/task_1.
    [Task 1] Finished. Success Rate: 0.00 - ❌ FAILURE

>>> Starting Task 2: pick_up_the_black_bowl_from_table_center_and_place_it_on_the_plate
Saved videos to evaluation_videos/task_2.
    [Task 2] Finished. Success Rate: 0.00 - ❌ FAILURE

>>> Starting Task 3: pick_up_the_black_bowl_on_the_cookie_box_and_place_it_on_the_plate
Saved videos to evaluation_videos/task_3.
    [Task 3] Finished. Success Rate: 0.00 - ❌ FAILURE

>>> Starting Task 4: pick_up_the_black_bowl_in_the_top_drawer_of_the_wooden_

In [ ]:
"""Run full-suite evaluation and display per-task success rates."""
results = evaluate_full_suite(
    checkpoint_path='best_trial_26.pt',
    action_stats_path='action_stats.json',
    benchmark='libero_spatial',
    env_num=1,
    max_steps=300,
    seed=42,
    save_videos=False,
    video_dir='evaluation_videos',
    camera_height=128,
    camera_width=128,
    video_skip=1
)

print("\nPer-task success rates:")
for task_id, info in results["per_task"].items():
    print(f"- {task_id} ({info['name']}): {info['success_rate']:.2f}")

print(f"\nAverage success rate: {results['average_success_rate']:.2f}")